In [1]:
import pandas as pd
import numpy as np

In [7]:
AG1_Data = {'W': 450*9.81,
           'D' : 8.4,
           'cuerda': 0.24, 
           'a' : 5.6,
           'Cdo' : 0.007, 
           'long_efec': 3.68,
           'alpha_rotor': 30,
           'Voo_Vh': 2, 
           'theta': 0
            }

Ex_1 = ElemPala(AG1_Data, 500, 0.08, 0)

In [8]:
class ElemPala:
    """Clase que te permite obtener los parámetros diferenciales (Levantamiento, Arrastre, Torque)
       de cada elemento de una pala de un autogiro o helicóptero, así como el torque total de cada 
       elemento en las dos palas. También se puede obtener el número de revoluciones necesario para
       entrar en autorotación cuando el torque total es cero
    """
    
    def __init__(self, Data, omega, dr, hdens):
        self.omega = omega
        
        self.Data = Data
        self.W = Data['W']
        self.D = Data['D'] 
        self.cuerda = Data['cuerda']; 
        self.a = Data['a'] 
        self.Cdo = Data['Cdo']
        self.long_efec = Data['long_efec']
        self.alpha_rotor = Data['alpha_rotor']
        self.Voo_Vh = Data['Voo_Vh']
        self.theta = Data['theta']
        
        self.dr = dr
        self.num_elem = int(self.long_efec/self.dr)
        self.r_0 = self.D/2 - self.long_efec
        
        self.hdens = hdens
        self.dens   = 1.225*(1-self.hdens*(0.00357/518.4))**(1/0.235)
        self.dens_ratio = self.dens/1.225
        
        self.T = self.W/np.cos(np.deg2rad(self.alpha_rotor))  
        self.Voo = self.Voo_Vh * (self.T / (2 * self.dens * np.pi * (1/4) * (self.D)**2))**0.5
        
        self.list_r = [0 for x in range(self.num_elem)]
        self.list_alpha_fb = [0 for x in range(self.num_elem)] # alphas de los elementos de la pala que avanza
        self.list_alpha_bb = [0 for x in range(self.num_elem)] # alphas de los elementos de la para que retrocede
        self.list_VR_fb = [0 for x in range(self.num_elem)]; self.list_VR_bb = [0 for x in range(self.num_elem)]
        self.list_L_fb = [0 for x in range(self.num_elem)]; self.list_L_bb = [0 for x in range(self.num_elem)] 
        self.list_D_fb = [0 for x in range(self.num_elem)]; self.list_D_bb = [0 for x in range(self.num_elem)]
        self.list_Q_fb = [0 for x in range(self.num_elem)]; self.list_Q_bb = [0 for x in range(self.num_elem)]
        
        self.DF = pd.DataFrame({'Omega [rpm]': [0], 'Q total': [0]})
        self.Q = []
        
    def calc_radios(self):
        """
        Método para obtener el radio correspondiente a cada elemento
        Args: None
        Output: None
        """
        self.list_r = [round(r,2) for r in np.arange(self.r_0 + self.dr, self.D/2 + self.dr, self.dr)]
        
    def calc_VR(self):
        """Método para obtener VR en cada elemento para la pala que avanza y
           la que retrocede
           Args: None
           Output: None
        """
        i = 0
        for r in self.list_r:
            self.list_VR_fb[i] = self.Voo * np.cos(np.deg2rad(self.alpha_rotor)) + self.omega*r*(1/60)*(2*np.pi)
            self.list_VR_bb[i] = abs(self.Voo * np.cos(np.deg2rad(self.alpha_rotor)) - self.omega*r*(1/60)*(2*np.pi))
            i += 1
        
    def calc_alpha(self):
        """Método que nos permite obtener el ángulo de ataque de cada elemento en radianes
           Args: None
           Output: None
        """
        i = 0
        for r in self.list_r:
            self.list_alpha_fb[i] = np.arctan((self.Voo * np.sin(np.deg2rad(self.alpha_rotor)))/self.list_VR_fb[i]) + self.theta*(np.pi/180)
            self.list_alpha_bb[i] = np.arctan((self.Voo * np.sin(np.deg2rad(self.alpha_rotor)))/self.list_VR_bb[i]) + self.theta*(np.pi/180)
            i += 1
    
    def calc_delta_L(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.list_VR_fb:
            self.list_L_fb[i] = 0.5*self.dens*(VR**2)*self.cuerda*self.a*self.list_alpha_fb[i]*self.dr
            i += 1
            
        i = 0
        for VR in self.list_VR_bb:
            self.list_L_bb[i] = 0.5*self.dens*(VR**2)*self.cuerda*self.a*self.list_alpha_bb[i]*self.dr
            i += 1
            
    def calc_delta_D(self):
        """Método para calcular el delta de Levantamiento en cada elemento
        """
        i = 0
        for VR in self.list_VR_fb:
            self.list_D_fb[i] = 0.5*self.dens*(VR**2)*self.cuerda*self.Cdo*self.dr
            i += 1
            
        i = 0
        for VR in self.list_VR_bb:
            self.list_D_bb[i] = 0.5*self.dens*(VR**2)*self.cuerda*self.Cdo*self.dr
            i += 1
        
    
    def calc_torque(self):
        """Método para calcular el torque total de ambas palas
        """
        for i in range(len(self.list_VR_fb)):
            self.list_Q_fb[i] = (self.list_D_fb[i] - self.list_alpha_fb[i]*self.list_L_fb[i])*self.list_r[i]*self.dr
            self.list_Q_bb[i] = (self.list_D_bb[i] - self.list_alpha_bb[i]*self.list_L_bb[i])*self.list_r[i]*self.dr
        
        self.Q = sum(self.list_Q_fb) + sum(self.list_Q_bb)
    
    def calc_All(self):
        """Método que calcula todos los parámetros necesarios para obtener el torque
        """
        self.__init__(self.Data, self.omega, self.dr, self.hdens)
        self.calc_radios()
        self.calc_VR()
        self.calc_alpha()
        self.calc_delta_L()
        self.calc_delta_D()
        self.calc_torque()
    
    def get_DataFrame_All(self):
        """Método que calcula todos los parámetros de todos los elementos de la pala y los pone en un Data Frame de pandas
        """
        self.calc_All()
        
        DF_total = pd.DataFrame({'r [m]': self.list_r,
                                'VR fb [m/s]': self.list_VR_fb,
                                'VR bb [m/s]': self.list_VR_bb,
                                'alpha fb [°]': np.degrees(self.list_alpha_fb),
                                'alpha bb [°]': np.degrees(self.list_alpha_bb),
                                'dL fb [N]': self.list_L_fb,
                                'dL bb [N]': self.list_L_bb,
                                'dD fb [N]': self.list_D_fb,
                                'dD bb [N]': self.list_D_bb 
                               })
        
        return DF_total
    
    def calc_list_torque(self, list_omega):
        """Método que calcula el torque para una lista de distintas rpm
           Inputs:
                    list_omega: lista. Lista de diferentes rpm para calcular su torque
           Outputs:
                   Data Frame de rpm y su respectivo torque
        """
        list_Q = [0 for x in range(len(list_omega))]
        
        i = 0
        for omega in list_omega:
            self.omega = omega
            self.calc_All()
            list_Q[i] = self.Q
            i += 1
        
        DF_Q = pd.DataFrame({'Omega [rpm]': list_omega, 'Torque [Nm]': list_Q})
        
        return DF_Q

In [9]:
AG1_Data = {'W': 450*9.81,
           'D' : 8.4,
           'cuerda': 0.24,
           'a': 5.6,
           'Cdo' : 0.007, 
           'long_efec': 3.68,
           'alpha_rotor': 30,
           'Voo_Vh': 2, 
           'theta': 0
            }

In [82]:
torque = [x for x in range(300, 600, 50)]
AG_EX1.calc_list_torque(torque)

,Omega [rpm],Torque [Nm]
0,300,-30.063787
1,350,-25.425146
2,400,-20.016561
3,450,-13.857705
4,500,-6.957704
5,550,0.678762


In [21]:
AG_EX1 = ElemPala(AG1_Data, 546.85, 0.08, 0)
AG_EX1.calc_All()

In [112]:
AG_EX1.T

5097.425526675205

In [113]:
delta_T = map(lambda x, y: x + y, AG_EX1.list_L_fb, AG_EX1.list_L_bb)

In [114]:
T_total = list(delta_T)
sum(T_total)

5097.669121883073

In [115]:
AG_EX1.Q

0.1759834746395268

In [49]:
import unittest

AG1_Data = {'W': 450*9.81,
           'D' : 8.4,
           'cuerda': 0.24,
           'a': 5.6,
           'Cdo' : 0.007, 
           'long_efec': 3.68,
           'alpha_rotor': 30,
           'Voo_Vh': 2, 
           'theta': 0
            }

class Test_bemClass(unittest.TestCase):
    def setUp(self):
        self.bem = ElemPala(AG1_Data, 500, 0.08, 0)
        self.bem.calc_All()
        
    def test_initializaion(self):
        #self.assertEqual(self.clothing.color, 'orange', 'color should be orange')
        self.assertEqual(round(self.bem.T,2), 5097.43, 'T = 5097.43 N')
        self.assertEqual(self.bem.list_r[4] , 0.92, 'Radio 5° Elem = 0.92 m')
        self.assertEqual(round(self.bem.Voo,2), 12.25, 'Voo = 12.25 m/s')
        self.assertEqual(round(self.bem.list_alpha_fb[4], 4) , 0.1039, 'Alpha 5° Elem fb = 0.1038 rad')
        self.assertEqual(round(self.bem.list_alpha_bb[4], 4) , 0.1617, 'Alpha 5° Elem bb = 0.1617 rad')
        self.assertEqual(round(self.bem.list_L_fb[4], 2) , 23.64, 'L 5° Elem fb = 23.64 N')
        self.assertEqual(round(self.bem.list_L_bb[4], 2) , 15.02, 'L 5° Elem bb = 15.02 rad')
        self.assertEqual(round(self.bem.list_D_fb[4], 2) , 0.28, 'D 5° Elem fb = 0.28 N')
        self.assertEqual(round(self.bem.list_D_bb[4], 2) , 0.12, 'D 5° Elem bb = 0.12 N')
        self.assertEqual(round(self.bem.list_Q_fb[4], 2) , -0.16, 'Q 5° Elem fb = -0.16 Nm')
        self.assertEqual(round(self.bem.list_Q_bb[4], 2) , -0.17, 'Q 5° Elem bb = -0.17 Nm')
        
tests = Test_bemClass()
tests_loaded = unittest.TestLoader().loadTestsFromModule(tests)
unittest.TextTestRunner().run(tests_loaded)

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>